In [ ]:
!pip install -q jedi

!pip install -q --upgrade \
    llama-index \
    llama-index-core \
    transformers \
    accelerate \
    bitsandbytes

!pip install -q \
    sentence-transformers \
    chromadb \
    llama-index-vector-stores-chroma \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    llama-index-retrievers-bm25


In [ ]:
import pkg_resources

packages = [
    "jedi",
    "llama-index",
    "llama-index-core",
    "transformers",
    "accelerate",
    "bitsandbytes",
    "sentence-transformers",
    "chromadb",
    "llama-index-vector-stores-chroma",
    "llama-index-llms-huggingface",
    "llama-index-embeddings-huggingface",
    "llama-index-retrievers-bm25"
]

for pkg in packages:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg} == {version}")
    except Exception as e:
        print(f"{pkg} ❌ NOT INSTALLED")


In [ ]:
import sys
import platform
import torch

# Python version
print("🐍 Python version:", sys.version)

# OS + môi trường
print("💻 OS:", platform.platform())
print("🏷️  Python executable:", sys.executable)

# Thử nhận dạng môi trường notebook
try:
    import google.colab
    print("🌐 Environment: Google Colab")
except:
    from IPython import get_ipython
    shell = get_ipython().__class__.__name__
    if shell == 'ZMQInteractiveShell':
        print("🧪 Environment: Jupyter / VSCode Notebook")
    elif shell == 'TerminalInteractiveShell':
        print("📟 Environment: Terminal / Local Python")
    else:
        print("❓ Environment: Unknown")

# Torch info
try:
    print("🔥 Torch version:", torch.__version__)
    print("⚡ CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("🔋 GPU name:", torch.cuda.get_device_name(0))
except:
    print("❌ Torch is not installed")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!hf auth login


In [ ]:
DOCX_PATH = "/content/drive/MyDrive/DTCNTT/DMS-5.pdf"
CACHE_FILE = "/content/drive/MyDrive/DTCNTT/data/cache/pipeline_cache.json"
INDEX_STORAGE = "/content/drive/MyDrive/DTCNTT/data/index_store_V6_maximalist"
COLLECTION_NAME = "dsm5_V6_store"

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


In [ ]:
# --- Các thư viện hệ thống & tiện ích ---
import traceback
import gc
import torch
import chromadb
torch.cuda.empty_cache()
# --- Hugging Face & Sentence Transformers ---
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import CrossEncoder

# --- LlamaIndex Core & LLM ---
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.schema import TextNode
from llama_index.core.llms import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer

# --- LlamaIndex Retrievers & Storage ---
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import VectorIndexRetriever, QueryFusionRetriever

In [ ]:

def setup_llm_and_embedding():
    global embed_model, llm, model

    # Dọn VRAM
    print("🧹 Clearing VRAM...")
    for obj in ["llm", "model", "embed_model"]:
        if obj in globals():
            del globals()[obj]
    gc.collect()
    torch.cuda.empty_cache()
    print("✅ VRAM cleared.\n")

    # ------------------------------
    # 1. Load Embedding
    # ------------------------------
    print("⏳ Loading embedding...")
    embed_model = HuggingFaceEmbedding(
        model_name="AITeamVN/Vietnamese_Embedding",
        device="cpu"
    )
    Settings.embed_model = embed_model
    print("✅ Embedding ready.\n")

    # ------------------------------
    # 2. Load Tokenizer từ HF HUB
    # ------------------------------
    REPO_ID = "letri345/llama3-8b-merge"  # <— sửa tên repo ở đây

    print(f"⏳ Loading tokenizer from {REPO_ID}...")
    tokenizer = AutoTokenizer.from_pretrained(REPO_ID, trust_remote_code=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Giữ template chat đã fine-tune
    tokenizer.chat_template = (
        "<|begin_of_text|>"
        "{% for message in messages %}"
        "{% if message['role'] == 'user' %}"
            "<|start_header_id|>user<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
        "{% elif message['role'] == 'system' %}"
            "<|start_header_id|>system<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
        "{% elif message['role'] == 'assistant' %}"
            "<|start_header_id|>assistant<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
        "{% endif %}"
        "{% endfor %}"
    )

    # ------------------------------
    # 3. Load Model 4-bit từ HF HUB
    # ------------------------------
    print("⚡ Loading 4-bit model from HuggingFace Hub...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        REPO_ID,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
    )

    print("✅ Model loaded successfully (4-bit).")

    # ------------------------------
    # 4. Wrap cho LlamaIndex
    # ------------------------------
    eos_id = tokenizer.eos_token_id
    eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer,
        context_window=8192,
        max_new_tokens=512,
        generate_kwargs={"temperature": 0.3},
        stopping_ids=[eos_id, eot_id],
    )

    Settings.llm = llm
    print("\n🎉 Llama 3 8B đã sẵn sàng sử dụng từ HuggingFace Hub!\n")


In [ ]:
setup_llm_and_embedding()

In [ ]:

db = chromadb.PersistentClient(path=INDEX_STORAGE)
col = db.get_collection(COLLECTION_NAME)
print("📦 Số lượng vectors trong Chroma:", col.count())


In [ ]:
def print_page(page_number):
    print(f"\n==================== PAGE {page_number} ====================\n")

    db = chromadb.PersistentClient(path=INDEX_STORAGE)
    col = db.get_collection(COLLECTION_NAME)

    results = col.get(
        where={"page_label": str(page_number)},
        include=["documents", "metadatas"]
    )

    if not results["documents"]:
        print("⚠️ Không tìm thấy trang này trong Chroma.")
        return

    text = results["documents"][0]
    meta = results["metadatas"][0]

    # In text
    print("📄 **NỘI DUNG TRANG:**")
    print("-" * 60)
    print(text[:1500])
    print("-" * 60)

    # In metadata
    print("\n🔖 **METADATA:**")
    for key, val in meta.items():
        print(f"  • {key}: {val}")

    # 🔢 Thêm phần đếm độ dài
    print("\n🔢 **ĐỘ DÀI DOCUMENT:**")
    print(f"  • Số ký tự: {len(text)}")
    print(f"  • Số từ: {len(text.split())}")

    print("\n============================================================\n")


In [ ]:
print_page(40)

In [ ]:
import re
import json
from llama_index.retrievers.bm25 import BM25Retriever

In [ ]:
def should_use_rag(message: str, last_user_message: str = "") -> bool:
    print("🧠 [Gate] Đang phân tích ý định (Expanded)...")

    msg = message.lower().strip()
    last_msg = last_user_message.lower().strip()

    # =================================================================
    # 🚨 NHÓM 0: KHỦNG HOẢNG & AN TOÀN (CRISIS)
    # Các từ khóa này báo hiệu nguy hiểm -> Cần xử lý ngay (Force RAG hoặc Alert)
    # =================================================================
    crisis_keywords = [
        "tự tử", "tự sát", "suicidal", "muốn chết", "kết thúc cuộc đời",
        "rạch tay", "tự hại", "self-harm", "nhảy lầu", "uống thuốc ngủ",
        "giết người", "hại người", "không muốn sống nữa", "tuyệt vọng"
    ]

    if any(k in msg for k in crisis_keywords):
        print("🧠 [Gate] ⚠️ PHÁT HIỆN TÍN HIỆU KHẨN CẤP.")
        return True

    # =================================================================
    # 📚 NHÓM 1: TÊN RỐI LOẠN (DISORDERS) - Mở rộng theo chương DSM-5
    # =================================================================
    disorders = [
        # --- Tâm trạng & Lo âu ---
        "trầm cảm", "depression", "u uất", "khí sắc",
        "lưỡng cực", "bipolar", "hưng cảm", "mania", "hypomania",
        "lo âu", "anxiety", "hoảng loạn", "panic", "sợ xã hội", "ám ảnh sợ",

        # --- OCD & Stress ---
        "ocd", "ám ảnh cưỡng bức", "nghi thức", "tích trữ", "hoarding",
        "ptsd", "sang chấn", "hậu chấn thương", "stress cấp tính", "thích ứng",

        # --- Loạn thần (Psychotic) ---
        "tâm thần phân liệt", "schizophrenia", "hoang tưởng", "ảo giác",
        "nghe tiếng nói", "loạn thần", "psychosis", "catatonia", "căng trương lực",

        # --- Phát triển thần kinh ---
        "tự kỷ", "autism", "asd", "tăng động", "giảm chú ý", "adhd",
        "khiếm khuyết trí tuệ", "chậm phát triển", "tik", "tourette",

        # --- Ăn uống (Feeding & Eating) ---
        "chán ăn", "anorexia", "ăn vô độ", "bulimia", "binge eating", "pica",

        # --- Nhân cách (Personality) ---
        "rối loạn nhân cách", "borderline", "ranh giới", "ái kỷ", "narcissistic",
        "chống đối xã hội", "antisocial", "tránh né", "phụ thuộc", "đa nhân cách",

        # --- Giấc ngủ & Khác ---
        "mất ngủ", "insomnia", "ngủ rũ", "narcolepsy", "ác mộng",
        "nghiện", "cai nghiện", "ma túy", "rượu", "chất kích thích",
        "rối loạn tình dục", "loạn dục", "giới tính", "dysphoria",
        "mất trí nhớ", "alzheimer", "giả bệnh"
    ]

    # =================================================================
    # 🩺 NHÓM 2: TRIỆU CHỨNG & BIỂU HIỆN (SYMPTOMS)
    # Những từ mô tả trạng thái bệnh lý cụ thể
    # =================================================================
    symptoms = [
        "ảo thanh", "ảo ảnh", "mất kiểm soát", "bốc đồng", "vô cảm",
        "kích động", "gây gổ", "thu mình", "sợ đám đông", "rửa tay liên tục",
        "kiểm tra liên tục", "nhớ lại", "flashback", "ác mộng",
        "mệt mỏi kinh niên", "sụt cân", "tăng cân", "mất ngủ kéo dài",
        "tim đập nhanh", "khó thở", "ngất", "run tay", "vã mồ hôi",
        "trống rỗng", "bỏ rơi", "đa nghi", "ghen tuông hoang tưởng"
    ]

    # =================================================================
    # 🔍 NHÓM 3: Ý ĐỊNH TRA CỨU (DIAGNOSTIC INTENT)
    # Các từ khóa thể hiện user muốn tìm kiến thức
    # =================================================================
    diagnostic_intent = [
        "tiêu chuẩn", "dấu hiệu", "biểu hiện", "triệu chứng",
        "chẩn đoán", "diagnose", "criteria", "xét nghiệm",
        "dsm", "dsm-5", "icd", "mã bệnh",
        "có phải là", "có phải bị", "tôi bị gì", "bệnh gì",
        "phân biệt", "khác nhau", "nguyên nhân", "yếu tố nguy cơ",
        "thời gian kéo dài", "bao lâu thì", "tiên lượng", "yếu tố"
    ]

    # --- LOGIC KIỂM TRA ---

    # 1. Check trực tiếp trong tin nhắn hiện tại
    # Gộp tất cả keywords để check 1 lần cho nhanh
    all_keywords = disorders + symptoms + diagnostic_intent

    # Dùng vòng lặp check từng từ (có thể tối ưu bằng regex nếu cần chính xác tuyệt đối)
    for kw in all_keywords:
        if kw in msg:
            print(f"🧠 [Gate] CẦN RAG (Keyword: '{kw}').")
            return True

    # 2. Check Context (Câu trước đó có nói về bệnh không?)
    # Nếu câu trước có keyword bệnh -> Câu này khả năng cao là follow-up
    context_keywords = disorders + symptoms # Chỉ quan tâm tên bệnh/triệu chứng ở context
    has_medical_context = any(kw in last_msg for kw in context_keywords)

    if has_medical_context:
        # Danh sách từ để NGẮT RAG (nếu user muốn dừng)
        stop_words = ["cảm ơn", "ok", "được rồi", "hiểu rồi", "bye", "tạm biệt", "không sao"]
        if any(w == msg for w in stop_words):
            print("🧠 [Gate] Context có bệnh, nhưng User dừng -> KHÔNG RAG.")
            return False

        # Nếu không phải từ dừng, mà context đang nói về bệnh -> Tiếp tục tra cứu
        print("🧠 [Gate] FORCE RAG (Theo ngữ cảnh hội thoại cũ).")
        return True

    # 3. Nếu không dính keyword nào -> Small Talk / General Chat
    print("🧠 [Gate] Không tìm thấy yếu tố chuyên môn -> KHÔNG RAG.")
    return False

In [ ]:
# Bạn cần import traceback nếu chưa có:
import traceback

def setup_chatbot_environment():
    """Load Embedding + LLaMA + Chroma + Hybrid RAG"""
    global memory, reranker_model, vector_index_chat, all_nodes
    global bm25_retriever, semantic_retriever, fusion_retriever, llm_chat
    global embed_model, llm  # Đảm bảo biến phase 1 truy cập được

    try:
        print("\n" + "="*60)
        print("🔹 GIAI ĐOẠN 2: SETUP CHATBOT HYBRID (sau reset kernel)")
        print("="*60 + "\n")

        # ---------------------------------------------------------
        # 🔥 KIỂM TRA – Giai đoạn 1 đã chạy chưa?
        # ---------------------------------------------------------
        if 'embed_model' not in globals() or embed_model is None:
            print("❗ embed_model chưa được load. Hãy chạy setup_llm_and_embedding() trước!")
            return False

        if 'llm' not in globals() or llm is None:
            print("❗ llm chưa được load. Hãy chạy setup_llm_and_embedding() trước!")
            return False

        # 1) Embedding cho query
        Settings.embed_model = embed_model

        # 2) LLaMA làm LLM chat
        llm_chat = llm
        Settings.llm = llm_chat

        # ---------------------------------------------------------
        # 3) Kết nối ChromaDB
        # ---------------------------------------------------------
        print(f"⏳ Đang kết nối ChromaDB tại: {INDEX_STORAGE}")
        db = chromadb.PersistentClient(path=INDEX_STORAGE)
        chroma_collection = db.get_collection(COLLECTION_NAME)

        print("📦 Số lượng vectors trong Chroma:", chroma_collection.count())

        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # ---------------------------------------------------------
        # 4) Load VectorStoreIndex từ Chroma
        # ---------------------------------------------------------
        vector_index_chat = VectorStoreIndex.from_vector_store(
            vector_store=vector_store,
            embed_model=embed_model,
        )
        print("✅ Vector Index đã loaqud từ Chroma.\n")

        # ---------------------------------------------------------
        # 5) LẤY NODES TRỰC TIẾP TỪ CHROMA (KHÔNG DÙNG docstore)
        # ---------------------------------------------------------
        print("⏳ Đang lấy nodes từ Chroma collection...")
        results = chroma_collection.get(
            include=["metadatas", "documents"]
        )

        all_nodes = []
        for doc, meta, _id in zip(results["documents"], results["metadatas"], results["ids"]):
            node = TextNode(
                text=doc,
                id_=_id,
                metadata=meta or {},
            )
            all_nodes.append(node)

        print(f"📦 Tổng số nodes reconstruct từ Chroma: {len(all_nodes)}\n")

        # ---------------------------------------------------------
        # 6) BM25 Retriever
        # ---------------------------------------------------------
        bm25_nodes = []
        for n in all_nodes:
            bm25_text = n.metadata.get("bm25_text", n.text)
            n_bm25 = TextNode(
                text=bm25_text,
                id_=n.node_id,
                metadata=n.metadata,
            )
            bm25_nodes.append(n_bm25)

        print("⏳ Tạo BM25Retriever (lexical)...")
        bm25_retriever = BM25Retriever.from_defaults(
            nodes=bm25_nodes,
            similarity_top_k=5,
        )
        print("✅ BM25Retriever sẵn sàng.\n")

        # ---------------------------------------------------------
        # 7) Semantic Retriever (vector)
        # ---------------------------------------------------------
        print("⏳ Tạo Semantic Retriever (vector)...")
        semantic_retriever = VectorIndexRetriever(
            index=vector_index_chat,
            similarity_top_k=5,
        )
        print("✅ Semantic Retriever sẵn sàng.\n")

        # ---------------------------------------------------------
        # 8) Fusion Retriever
        # ---------------------------------------------------------
        print("⏳ Tạo QueryFusionRetriever (Hybrid)...")
        fusion_retriever = QueryFusionRetriever(
            retrievers=[bm25_retriever, semantic_retriever],
            similarity_top_k=5,
            num_queries=2,
            mode="reciprocal_rerank",
        )
        print("✅ Hybrid Fusion Retriever sẵn sàng.\n")

        # ---------------------------------------------------------
        # 9) Reranker
        # ---------------------------------------------------------
        print("⏳ Tải Cross-Encoder Reranker...")
        # Đặt thiết bị mặc định là "cpu"
        reranker_model = CrossEncoder(
            "cross-encoder/ms-marco-MiniLM-L6-v2",
        )
        print("✅ Reranker đã sẵn sàng.\n")

        # ---------------------------------------------------------
        # 10) Memory + System Prompt
        # ---------------------------------------------------------
        SYSTEM_PROMPT = """\
BBạn là một **chuyên gia tâm lý AI** (Trợ lý) được phát triển bởi **AI VIETNAM**.
Nhiệm vụ của bạn là trò chuyện, theo dõi và tư vấn cho người dùng (User) về sức khỏe tâm thần.
Luôn luôn hành động với tư cách là Trợ lý, thể hiện sự đồng cảm và chuyên nghiệp.

===========================
QUY TẮC RAG:
===========================
- Nếu User hỏi về thông tin chuyên môn (triệu chứng, DSM-5, rối loạn, trị liệu, phân loại chẩn đoán...),
  bạn SẼ nhận được một tin nhắn System chứa thông tin tham khảo.
- Hãy dựa HOÀN TOÀN vào thông tin đó để trả lời.
- Tuyệt đối KHÔNG được tự bịa kiến thức khi đã kích hoạt RAG.

===========================
QUY TẮC KHÔNG RAG:
===========================
- Khi KHÔNG có RAG, bạn trò chuyện tự nhiên như một nhà trị liệu tâm lý:
  luôn đồng cảm, lắng nghe, phản ánh cảm xúc, đặt câu hỏi mở.
- Tuy nhiên, hãy **gợi ý một cách tinh tế** để người dùng chia sẻ sâu hơn —
  nhằm nhận biết khi nào nên chuyển sang RAG.
  Ví dụ:
  - “Bạn có cảm thấy triệu chứng đó kéo dài bao lâu rồi?”
  - “Điều này có ảnh hưởng mạnh đến sinh hoạt của bạn không?”
  - “Bạn mô tả rõ hơn cảm giác trong những lúc như vậy được không?”

  Tránh hỏi dồn dập; chỉ gợi mở nhẹ nhàng để nhận diện nhu cầu chuyên môn.

===========================
QUY TẮC BẢO VỆ (RẤT QUAN TRỌNG):
===========================
- Bạn **TUYỆT ĐỐI KHÔNG** trả lời các câu hỏi KHÔNG liên quan đến tâm lý hoặc sức khỏe tâm thần.
- Các chủ đề CẤM gồm nhưng không giới hạn:
  **thời tiết, nấu ăn, chính trị, thể thao, tin tức, lập trình, toán học, công nghệ, mẹo đời sống, gossip...**
- Khi bị hỏi về chủ đề cấm, hãy lịch sự từ chối và kéo người dùng về đúng chủ đề.
  Ví dụ:
  “Tôi xin lỗi, tôi chỉ được đào tạo để hỗ trợ về sức khỏe tâm thần.
   Bạn có đang gặp khó khăn nào khiến bạn muốn trò chuyện hôm nay không?”

===========================
PHONG CÁCH TRẢ LỜI:
===========================
- Luôn giữ giọng nói:
  **ấm áp – không phán xét – chuyên nghiệp – nhẹ nhàng – hướng dẫn.**
- Ngắn gọn nhưng sâu sắc.
- Không bao giờ biến thành chatbot đa năng.
- Bạn là một **therapist AI**, không phải “trợ lý hỏi gì cũng trả lời”.

"""

        memory = ChatMemoryBuffer.from_defaults(token_limit=4096)
        memory.put(ChatMessage(role="system", content=SYSTEM_PROMPT))

        print("🎉 Giai đoạn 2: Setup HYBRID hoàn tất.\n")
        return True

    except Exception as e:
        print(f"❌ LỖI GIAI ĐOẠN 2 (SETUP): {e}")
        traceback.print_exc()
        return False


In [ ]:
def rag_retrieve_and_rerank(message: str, top_k=3, final_top=2) -> str:
    print(f"🔍 [RAG HYBRID] Truy vấn: '{message}'")

    # 1) Hybrid retrieve
    retrieved_docs = fusion_retriever.retrieve(message)
    if not retrieved_docs:
        print("⚠️ Không tìm thấy tài liệu liên quan.")
        return ""

    print(f"🔍 [RAG HYBRID] Fusion trả về {len(retrieved_docs)} docs. Đang rerank...")

    # Chỉ rerank tối đa 6 documents để tăng tốc
    retrieved_docs = retrieved_docs[:6]

    # 2) Chuẩn bị input cho CrossEncoder
    doc_texts = [doc.get_content() for doc in retrieved_docs]
    pairs = [(message, text) for text in doc_texts]

    # 3) Rerank
    try:
        scores = reranker_model.predict(pairs)
        ranked = sorted(
            zip(scores, retrieved_docs),
            key=lambda x: x[0],
            reverse=True,
        )
        ranked_docs = [doc for _, doc in ranked]
    except Exception as e:
        print(f"❌ Rerank lỗi: {e}")
        ranked_docs = retrieved_docs

    # 4) Top N cuối cùng
    top_docs = ranked_docs[:final_top]
    context_text = "\n\n---\n\n".join([doc.get_content() for doc in top_docs])

    # 5) Tóm tắt nếu quá dài
    if len(context_text.split()) > 700:
        print("✂️ Context quá dài – đang tóm tắt (dùng llm_chat)...")
        try:
            summary_messages = [
                ChatMessage(
                    role="system",
                    content=(
                        "Bạn là AI tóm tắt tài liệu DSM-5. "
                        "Tóm tắt ngắn gọn, chính xác, chỉ giữ phần liên quan tới câu hỏi."
                    ),
                ),
                ChatMessage(
                    role="user",
                    content=f"Câu hỏi: {message}\n\nNội dung tài liệu:\n{context_text}",
                ),
            ]
            summary_response = llm_chat.chat(summary_messages)
            context_text = summary_response.message.content.strip()
        except Exception as e:
            print(f"❌ Lỗi tóm tắt: {e}")
            context_text = " ".join(context_text.split()[:700])

    return f"--- Thông tin tham khảo từ DSM-5 ---\n{context_text}\n--- Hết thông tin tham khảo ---"


In [ ]:
setup_chatbot_environment()

In [ ]:
# =====================================================
# 🎯 RESET MEMORY – CHUẨN HÓA CHO ChatMemoryBuffer
# =====================================================
def reset_memory():
    """
    Reset bộ nhớ hội thoại cho ChatMemoryBuffer trong LlamaIndex.
    Dựa đúng cấu trúc memory.__dict__ mà bạn đang dùng.
    """
    try:
        memory.chat_store.store["chat_history"] = []
        print("💾 Bộ nhớ đã được reset.")
    except Exception as e:
        print(f"⚠️ Không thể reset bộ nhớ: {e}")


# =====================================================
# 🎯 HÀM GỌI MODEL (TIỆN ÍCH)
# =====================================================
def call_llm(messages):
    """Helper gọi LLaMA + sanitize output."""
    response = llm_chat.chat(messages)
    text = response.message.content.strip()

    if text.lower().startswith("assistant"):
        text = text[len("assistant"):].lstrip(":").strip()

    if not text:
        text = (
            "Mình chưa rõ lắm, "
            "bạn có thể mô tả rõ hơn cảm xúc hoặc vấn đề của bạn được không?"
        )

    return text


# =====================================================
# 🎯 VÒNG LẶP CHAT TỐI ƯU (ĐÃ FIX)
# =====================================================
def chat_conversation():
    print("\n" + "="*60)
    print("🤖 Chatbot DSM-5 HYBRID đã sẵn sàng. Gõ 'quit' để thoát.")
    print("="*60 + "\n")

    while True:
        try:
            user_message = input("Bạn: ").strip()
            if not user_message:
                continue

            if user_message.lower() in ["quit", "exit"]:
                print("Chatbot: Cảm ơn bạn đã trò chuyện. Hẹn gặp lại 🌷")
                break

            new_user_message = ChatMessage(role="user", content=user_message)
            messages_to_send = memory.get_all()

            # =====================================================
            # 🔍 1) TRUY VẤN META RAG → CẤM, RESET, END
            # =====================================================
            forbidden_meta = ["rag", "retrieve", "rerank", "vector", "dsm context"]
            if any(key in user_message.lower() for key in forbidden_meta):

                print("Chatbot: Tôi không thể trả lời truy vấn này. "
                      "Vui lòng đặt câu hỏi liên quan đến sức khỏe tâm thần.\n")

                reset_memory()
                break


            # =====================================================
            # 🔍 2) KÍCH HOẠT RAG
            # =====================================================
            if should_use_rag(user_message):

                context = rag_retrieve_and_rerank(user_message)

                if context:
                    messages_to_send.append(
                        ChatMessage(role="system", content=context)
                    )
                else:
                    messages_to_send.append(
                        ChatMessage(
                            role="system",
                            content=(
                                "--- Thông tin tham khảo từ DSM-5 ---\n"
                                "Không tìm thấy đoạn phù hợp.\n"
                                "--- Hết thông tin tham khảo ---"
                            )
                        )
                    )

                messages_to_send.append(new_user_message)
                response_text = call_llm(messages_to_send)

                print(f"Chatbot: {response_text}\n")

                reset_memory()
                print("💾 Bộ nhớ đã được reset sau truy vấn RAG.\n")
                continue


            # =====================================================
            # 🔍 3) HỘI THOẠI THƯỜNG
            # =====================================================
            messages_to_send.append(new_user_message)
            response_text = call_llm(messages_to_send)

            memory.put(new_user_message)
            memory.put(ChatMessage(role="assistant", content=response_text))

            print(f"Chatbot: {response_text}\n")

        except KeyboardInterrupt:
            print("\n🛑 Đã dừng cuộc trò chuyện.")
            break

        except Exception as e:
            print(f"❌ Lỗi không mong muốn trong vòng lặp chat: {e}")
            traceback.print_exc()
            break


In [ ]:
chat_conversation()

In [ ]:

import os
import openai


client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# ----------------------------
# Load GPT-4o-mini
# ----------------------------
llm_gptmini = OpenAI(
    model="gpt-4o-mini",
    temperature=0.2,   # giảm độ sáng tạo, tập trung fact-based
    max_tokens=512
)

# Gán cho Settings để dùng trong ingest + transformation
Settings.llm_gptmini = llm_gptmini

# Nếu muốn dùng GPT cho đánh giá câu hỏi cũng lấy llm này


In [ ]:
import json
import pandas as pd

In [ ]:
def generate_questions_from_nodes(nodes, max_q_per_node=1):
    questions = []
    for node in nodes:
        prompt = f"""
Bạn là giảng viên tâm lý. Dựa vào đoạn văn bản DSM-5:
"{node.text[:1000]}"

Soạn {max_q_per_node} câu hỏi tự luận (Essay):
- Không Yes/No
- Ngắn gọn, tổng hợp thông tin
- Trả về JSON List: ["Câu hỏi 1", ...]
"""
        resp = Settings.llm_gptmini.chat([ChatMessage(role="user", content=prompt)])
        content = resp.message.content.strip()

        # Clean JSON nếu có Markdown
        if content.startswith("```json"):
            content = content.replace("```json"," ").replace("```"," ")
        try:
            q_list = json.loads(content)
            for q in q_list:
                questions.append({"question": q, "reference_node_text": node.text})
        except:
            print(f"⚠️ Node {node.node_id} không parse được JSON.")
    return pd.DataFrame(questions)

In [ ]:
def answer_with_llama(question, top_k=3, final_top=2):
    """
    Trả lời một câu hỏi theo đúng logic của chat pipeline:
    - Nếu cần RAG thì thêm context + reset memory sau khi trả lời
    - Nếu không cần RAG thì trả lời trực tiếp
    - KHÔNG ghi vào bộ nhớ hội thoại
    """
    # Detect cấm/meta
    forbidden_meta = ["rag", "retrieve", "rerank", "vector", "dsm context"]
    if any(k in question.lower() for k in forbidden_meta):
        return "❌ Câu hỏi chứa từ khóa hệ thống – không hợp lệ."

    # Nếu cần RAG
    if should_use_rag(question):
        context = rag_retrieve_and_rerank(question, top_k=top_k, final_top=final_top)

        system_msg = (
            f"--- Thông tin DSM-5 liên quan ---\n{context}\n"
            "--- Hết thông tin ---"
        )

        messages = [
            ChatMessage(role="system", content=system_msg),
            ChatMessage(role="user", content=question)
        ]

        answer = call_llm(messages)

        # QUY TẮC BẮT BUỘC CỦA BẠN
        reset_memory()

        return answer

    # Câu hỏi bình thường
    messages = [
        ChatMessage(role="system", content="Bạn là trợ lý tâm lý AI."),
        ChatMessage(role="user", content=question)
    ]
    return call_llm(messages)


In [ ]:
def evaluate_with_gpt(question, answer, reference_text=None):
    """
    Đánh giá câu trả lời GPT dựa trên câu hỏi và văn bản tham chiếu (nếu có).
    Trả điểm dạng float để đánh giá mượt hơn.
    """

    # Cắt theo chunk size nếu có reference
    ref_text_snippet = reference_text[:1000] if reference_text else None

    eval_prompt = f"""
Bạn là giám khảo tâm lý AI. Hãy đánh giá câu trả lời sau:

Question: "{question}"
Answer: "{answer}"
"""
    if ref_text_snippet:
        eval_prompt += f"""
Văn bản tham chiếu (reference):
"{ref_text_snippet}"

Hãy chấm điểm dựa trên việc câu trả lời có đúng và trung thực với reference_text.
"""

    eval_prompt += """
TIÊU CHÍ (dạng FLOAT):
1. Correctness: 0.0–5.0
2. Faithfulness: 0.0–1.0
3. Relevancy: 0.0–1.0

Trả về JSON:
{"correctness": float, "faithfulness": float, "relevancy": float}
"""

    # GPT trả lời
    resp = Settings.llm_gptmini.chat([ChatMessage(role="user", content=eval_prompt)])
    content = resp.message.content.strip()

    # Clean JSON nếu có Markdown
    if content.startswith("```"):
        content = content.replace("```json", "").replace("```", "")

    # Parse JSON
    try:
        data = json.loads(content)

        # Convert tất cả sang float an toàn
        result = {
            "correctness": float(data.get("correctness", 0)),
            "faithfulness": float(data.get("faithfulness", 0)),
            "relevancy": float(data.get("relevancy", 0))
        }
        return result

    except Exception:
        return {
            "correctness": 0.0,
            "faithfulness": 0.0,
            "relevancy": 0.0,
            "reasoning": "Parse error"
        }


In [ ]:
# ===============================================
# 5️⃣ In điểm trung bình
# ===============================================
def print_average_scores(df_eval):
    print("📊 Điểm trung bình:")
    print(f"Correctness: {df_eval['correctness'].mean():.2f} / 5")
    print(f"Faithfulness: {df_eval['faithfulness'].mean():.2f} / 1")
    print(f"Relevancy: {df_eval['relevancy'].mean():.2f} / 1")


In [ ]:
# 5️⃣ Lưu kết quả ra CSV
# ===============================================
def save_eval_results(df_eval, filename="eval_results.csv"):
    df_eval.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"💾 Kết quả đánh giá đã lưu: {filename}")

In [ ]:
from tqdm import tqdm

def generate_question_set(nodes, max_q_per_node=1, save_path="questions_full.csv"):
    """
    Sinh toàn bộ câu hỏi từ nodes với tqdm quan sát tiến trình.
    """
    print("\n🧩 Đang sinh toàn bộ câu hỏi từ nodes...")

    all_questions = []

    # tqdm để theo dõi từng node
    for node in tqdm(nodes, desc="Generating questions"):
        df_node = generate_questions_from_nodes(
            [node],
            max_q_per_node=max_q_per_node
        )
        all_questions.append(df_node)

    # Gộp tất cả
    df_questions_all = pd.concat(all_questions, ignore_index=True)

    print(f"📦 Tổng câu hỏi được sinh ra: {len(df_questions_all)}")

    # Lưu CSV
    if save_path:
        try:
            df_questions_all.to_csv(save_path, index=False, encoding='utf-8')
            print(f"💾 Đã lưu toàn bộ câu hỏi tại: {save_path}")
        except Exception as e:
            print("❌ Lỗi khi lưu CSV:", e)

    return df_questions_all


In [ ]:
def evaluate_question_set(csv_path, max_questions=200):
    """
    Đánh giá câu hỏi từ CSV:
    - Load toàn bộ câu hỏi
    - Sample max_questions câu
    - GPT trả lời
    - GPT tự đánh giá dựa reference_text
    """

    print("\n🔎 Bắt đầu load câu hỏi từ CSV...")

    # 1) Load CSV
    try:
        df_questions_full = pd.read_csv(csv_path)
    except Exception as e:
        print(f"❌ Không thể load CSV: {e}")
        return None

    print(f"📦 Tổng câu hỏi trong CSV: {len(df_questions_full)}")

    # 2) Random sample lúc đánh giá
    if len(df_questions_full) > max_questions:
        df_questions = (
            df_questions_full
            .sample(n=max_questions, random_state=42)
            .reset_index(drop=True)
        )
    else:
        df_questions = df_questions_full.copy()

    print(f"📘 Số câu hỏi đem đi đánh giá: {len(df_questions)}")

    results = []

    # 3) Đánh giá từng câu với progress bar
    for idx, row in tqdm(df_questions.iterrows(),
                         total=len(df_questions),
                         desc="Evaluating questions"):
        question = row["question"]
        reference_text = row["reference_node_text"]

        # LLAMA trả lời
        answer = answer_with_llama(question)

        # GPT đánh giá
        eval_res = evaluate_with_gpt(
            question=question,
            answer=answer,
            reference_text=reference_text
        )

        eval_res.update({
            "question": question,
            "answer": answer,
            "reference_text": reference_text
        })

        results.append(eval_res)

    # 4) Convert kết quả
    df_result = pd.DataFrame(results)

    # 5) In điểm trung bình
    if not df_result.empty:
        print_average_scores(df_result)

    # 6) Lưu kết quả ra file
    save_eval_results(df_result)

    print("🎉 Đánh giá hoàn tất!")
    return df_result

In [ ]:
!pip install -q nest_asyncio

In [ ]:
df_questions_all = generate_question_set(
    all_nodes,
    max_q_per_node=1,
    save_path="questions_full.csv"
)


In [ ]:
df_eval = evaluate_question_set(
    csv_path="/content/drive/MyDrive/DTCNTT/questions_full.csv",
    max_questions=200
)


In [ ]:
print("\n🔹 Sample kết quả:")
print(df_eval.head())

In [ ]:
print_average_scores(df_eval)

In [ ]:
# 1️⃣ Thông tin tổng quan
print("=== Thông tin DataFrame ===")
print(df_eval.info())
print("\n=== Thống kê mô tả các cột số ===")
print(df_eval.describe())

In [ ]:
sample_df = df_eval[['question', 'answer', 'correctness', 'faithfulness', 'relevancy']].sample(10, random_state=42)
sample_df